In [1]:
ensorflow as tf
frimport tom tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [2]:
# these are just variables for using while tokenization,setting data usage limits and other purposes
training_size = 15000
embedding_dim = 16
vocab_size = 10000
max_length = 100
trunc_type='post'
padding_type='post'

In [3]:
df=pd.read_csv("train.csv")
#reading data from a csv file dataset
print(df.columns)#shows all the columnnames of the dataset 
print(df.head(5))#shows data of the first five rows
df.dropna(inplace=True)#removes the missing value rows
print(df.isnull().sum())#shows missing value counts of each attribute
print(df.shape)#shows the dimensions of the dataset


text= []
label = []

#storing the title and label values in the empty lists
ti= df.loc[:,'text']
text= ti.values.tolist()
for i in range(len(text)):
    x=isinstance(text[i],str)
    if x == False:
        text[i] = str(text[i])
        

lbl= df.loc[:,'label']
label= lbl.values.tolist()

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')
   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  
id        0
title     0
author    0
text      0
label     0
dtype: int64
(18285, 

In [4]:
#separates the training and testing titles for validation
training_title = text[0:training_size]
testing_title = text[training_size:]

#separates the training and testing labels for validation
training_label = label[0:training_size]
testing_label = label[training_size:]

In [6]:
# this section adds tokens to every word found after fitting on the training sentences
tokenizer = Tokenizer(num_words=vocab_size,oov_token="<oov>")
tokenizer.fit_on_texts(training_title)

# holds the index value of each word found on in the training data sets
word_index = tokenizer.word_index

# text to sequences is a procedure to convert the words of each sentences into sequence represented by a list
# padding is a procedure of adding zeros to empty spaces just to maintain a constant length of every sentence
training_sequences = tokenizer.texts_to_sequences(training_title)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_title)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# print(training_sequences)
# print(training_padded)

#storing the padded_titles and labels in arrays
training_padded_title = np.array(training_padded)
testing_padded_title = np.array(testing_padded)

training_label = np.array(training_label)
testing_label = np.array(testing_label)

# model creation code is below. (Sequential) allows us to create multiple neural networks. (Embedding) is a process of graphing the sentiment of each word, epoch by epoch, on a numberline(vector) and then applying multiple vectors for multiple words. 
# (AveragePooling) provides the resultant vector sum value by a binary sentiment like good or bad.
# these values are then fed to a deep neural network
# RELU stand for rectified linear unit which is simply an if else condition which returns a value if it is (>) greater than 0 or returns 0.
# 24 and 1 are the number of neurons on a neural network
# the activation function specifies what is the work of each neural network
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
# NOTE: 'Loss' provides the calculation of loss prediction value and 'optimizer' is the error rectification procedure through their respective functions 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()
num_epochs = 50

# training of the model is done using the model.fit
title_history = model.fit(training_padded_title, training_label, epochs=num_epochs, validation_data=(testing_padded_title,testing_label), verbose=2)
title_history

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________
Train on 15000 samples, validate on 3285 samples
Epoch 1/50
15000/15000 - 3s - loss: 0.4416 - accuracy: 0.7989 - val_loss: 0.2282 - val_accuracy: 0.9205
Epoch 2/50
15000/15000 - 2s - loss: 0.1532 - accuracy: 0.9449 - val_loss: 0.1595 - val_accura

In [10]:
news = []
print('Enter number of news to be validated' )
#taking the number of news to be validate as an integer input from the user
news_number= int(input())
for i in range(news_number):
    print('Enter news to be validated')
    #taking news input as string from the user
    n = input()
    news.append(n)
print(news)

#tokenization of the input news
sequences = tokenizer.texts_to_sequences(news)
#padding of the tokenized sequences
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#predicting a value for evaluation
predict_value = model.predict(padded)
print(predict_value)
for i in range(len(news)):
    print(predict_value[i])
    if predict_value[i] > 0.9996:
        print(news[i]+' IS A TRUE NEWS')
    else:
        print(news[i]+' IS A FAKE NEWS')

Enter number of news to be validated
2
Enter news to be validated
New york in in India
Enter news to be validated
Sun rises in west
['New york in in India', 'Sun rises in west']
[[0.9987527 ]
 [0.99460727]]
[0.9987527]
New york in in India IS A FAKE NEWS
[0.99460727]
Sun rises in west IS A FAKE NEWS
